# Information Retrieval

This notebook contains some experiments with information retrieval.
Topics:
* indexing
* search
* result scoring: TF-IDF, vector space models, term frequency counts, etc.

In [1]:
from matplotlib import pyplot as plt
from datetime import datetime
from collections import defaultdict, Counter
import itertools
import hashlib
import urllib.request
import itertools
import re
import numpy as np
import random
import math
import torch
import uuid

In [2]:
"""Shakespeare's sonnets"""

ROMAN_NUMERAL = re.compile(r'^M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$')
MD5_DIGEST = '83ad11c60641bb60d44fb75c406c0c9b'

def download_sonnets():
    raw = urllib.request.urlopen('https://www.gutenberg.org/cache/epub/1041/pg1041.txt').read()
    assert hashlib.md5(raw).hexdigest() == MD5_DIGEST, "md5 does not match"
    return raw.decode('utf-8')

def extract_sonnet(lines, start):
    num = lines[start].strip()
    title = f'Sonnet {num}'
    content = []
    for pos in range(start+2, len(lines)):
        line = lines[pos].strip()
        if not line:
            break
        content.append(line)
    return title, "\n".join(content)

def extract_sonnets(content):
    lines = content.split('\n')
    starts = [pos for pos, line in enumerate(lines) if line.strip() and ROMAN_NUMERAL.match(line.strip())]
    sonnets = []
    for start in starts:
        sonnets.append(extract_sonnet(lines, start))
    return sonnets

# [(title, content)]
sonnets = extract_sonnets(download_sonnets())

In [3]:
# Words for document generation
words = open('/usr/share/dict/words').read().splitlines()

In [4]:
"""Doc generation"""
def sentence(n): return ' '.join(random.choice(words) for _ in range(n)) + '.'
def content(n): return ' '.join(sentence(10) for _ in range(n // 10))
def mktitle(): return ' '.join(random.choice(words) for _ in range(5))
def mkauthor(): return random.choice(words) + ' ' + random.choice(words)
def mkdate(): return datetime.utcnow()
gdocids = itertools.count()
def mkdoc(**fields): return {
    'docid': next(gdocids),
    'title': mktitle(),
    'author': mkauthor(),
    'date': mkdate(),
    'content': content(random.randint(64, 128)),
    **fields
}
def mkdocs(n): return [mkdoc() for _ in range(n)]

In [5]:
sonnet_docs = [mkdoc(title=title, content=content, author='shakespeare') for title, content in sonnets]
rand_docs = mkdocs(16)

In [6]:
"""Indexing"""

def tokenize(s): return (tok.lower() for tok in re.findall(r'\w+', s))

def index_doc(doc):
    """Returns {term: [offset]}"""
    index = defaultdict(list)
    for pos, term in enumerate(tokenize(doc['content'])):
        index[term].append(pos)
    return index

def rev_index(docs):
    """Returns {term: [docid]}"""
    index = defaultdict(list)
    for doc in docs:
        for term in tokenize(doc['content']):
            postings = index[term]
            if not postings or postings[-1] != doc['docid']:
                postings.append(doc['docid'])
    return index

In [7]:
sonnet_rindex = rev_index(sonnet_docs)
sonnet_doc_indexes = [index_doc(doc) for doc in sonnet_docs]

In [10]:
"""Searching"""
import bisect

def isect(left, right):
    """Intersect two sorted iterables"""
    left, right = iter(left), iter(right)
    l, r = next(left, None), next(right, None)
    while l is not None and r is not None:
        if l == r:
            yield l
            l, r = next(left, None), next(right, None)
        elif l < r:
            l = next(left, None)
        else:
            r = next(right, None)
            
def isectn(*lists):
    """Intersect N sorted iterables"""
    it = iter(lists)
    res = next(it, [])
    for xs in it:
        res = isect(res, xs)
    return res
    
def exact_search(docindex, query):
    """Returns [term_offset] of exact matches"""
    if not query: 
        return []
    hits = [docindex[term] for term in query]
    idx = 0
    results = []
    while idx < len(hits[0]):
        offset = hits[0][idx]
        next_offset = None
        for j in range(1, len(query)):
            jhits = hits[j]
            jidx = bisect.bisect_left(jhits, offset)
            if jidx == len(jhits):
                return results
            joffset = jhits[jidx]
            if joffset != offset + j:
                next_offset = joffset - j
                break
        if next_offset is None:
            results.append(offset)
            idx += 1
        else:
            idx = bisect.bisect_left(hits[0], next_offset)
    return results
    
    
def search(rindex, query):
    doclists = [rindex[term] for term in query]
    return list(isectn(*doclists))

def parse_query(query):
    return list(tokenize(query))

def pretty_search(docs, rindex, query):
    titles = {doc['docid']: doc['title'] for doc in docs}
    results = search(rindex, parse_query(query))
    return [titles[docid] for docid in results]


In [11]:
"""A few tests never hurt"""
assert list(isect([0, 1, 2], [-1, 1, 3])) == [1]
assert list(isectn([0, 1, 2], [1], [-1, 1, 2, 5])) == [1]
rindex = {
    'a': [0, 1, 2],
    'b': [1],
    'c': [1, 2],
}
assert search(rindex, ['a']) == [0, 1, 2]
assert search(rindex, ['a', 'c']) == [1, 2], search(rindex, ['a', 'c'])
assert search(rindex, ['a', 'b', 'c']) == [1]

def test_exact_search():
    sample = """
jack and jill ran up the hill
but no one stopped to pay the bill
up the hill up the jack the hill jack and jill
"""
    queries = [
        'jack',
        'jack and jill',
        'no one stopped',
        'up the hill',
        'FOO BAR'
    ]
    tokens = list(tokenize(sample))
    dindex = index_doc(mkdoc(content=sample))
    for q in queries:
        pq = parse_query(q)
        expect = [offset for offset, _ in enumerate(tokens) if tokens[offset:offset+len(pq)] == pq]
        assert exact_search(dindex, pq) == expect

test_exact_search()

In [12]:
queries = [
    "fairest creatures",
    "her beauty",
    "spring",
    "tender",
    "thine own",
]
for q in queries:
    print(f"{q}: {pretty_search(sonnet_docs, sonnet_rindex, q)}")


fairest creatures: ['Sonnet I']
her beauty: ['Sonnet IX', 'Sonnet XI', 'Sonnet XIX', 'Sonnet XXII', 'Sonnet XLI', 'Sonnet CXXVII']
spring: ['Sonnet I', 'Sonnet LIII', 'Sonnet LXIII', 'Sonnet XCVIII', 'Sonnet CII']
tender: ['Sonnet I', 'Sonnet XXII', 'Sonnet XLV', 'Sonnet LXXXIII', 'Sonnet CXX', 'Sonnet CXXVIII', 'Sonnet CXLI']
thine own: ['Sonnet I', 'Sonnet II', 'Sonnet XIX', 'Sonnet XXXVIII', 'Sonnet XLIX', 'Sonnet LXIX', 'Sonnet CXLII']


In [27]:
"""Scoring"""

def docfreqs(docs):
    docfreqs = Counter()
    for doc in docs:
        for term in set(tokenize(doc['content'])):
            docfreqs[term] += 1
    return docfreqs

def termfreqs(doc):
    return Counter(tokenize(doc['content']))
    
def tfidf(docs):
    df = docfreqs(docs)
    for doc in docs:
        scores = {}
        for term, tf in termfreqs(doc).items():
            idf = math.log(len(docs) / df[term], 2)
            tfidf = tf * idf
            scores[term] = tfidf
        yield scores
        
def tfidf_table(docs):
    return {doc['docid']: scores for doc, scores in zip(docs, tfidf(docs))}

def score_tfidf(doc, query):
    pass  # TODO: implement
   
def score_freq(doc_index, query):
    return sum(len(doc_index[term]) for term in query)

def close_hits(doc_index, query, threshold):
    hits = [doc_index[term] for term in query]
    for idx in range(len(hits[0])):
        offset = hits[0][idx]
        indices = [idx] + [bisect.bisect_left(hits[j], offset) for j in range(1, len(hits))]
        offsets = [hits[j][idx] if idx < len(hits[j]) else -1 for j, idx in enumerate(indices)]
        for j in range(1, len(offsets)):
            if offsets[j] == -1 or offsets[j-1] == -1:
                continue
            distance = offsets[j] - offsets[j-1]
            if distance <= threshold:
                yield distance
    
def score_proximity(doc_index, query, prox_bins=None, count_bins=None):
    if not query:
        return 0.0
    prox_bins = prox_bins or [(0, 1.0), (1, 0.9), (2, 0.8), (3, 0.7), (4, 0.6), (5, 0.5)]
    count_bins = count_bins or [(0, 0.0), (1, 0.5), (2, 0.75), (3, 0.85), (4, 0.95), (5, 1.0)]
    prox_counts = [0 for _ in prox_bins]
    threshold = max(prox for prox, weight in prox_bins)
    for distance in close_hits(doc_index, query, threshold=threshold):
        binidx = next(i for i in range(len(prox_bins)) if distance <= prox_bins[i][0])
        prox_counts[binidx] += 1
    prox_weights = [weight for prox, weight in prox_bins]
    count_weights = [next((weight for threshold, weight in count_bins if count <= threshold), 0.0)
                    for count in prox_counts]
    return np.dot(prox_weights, count_weights)


In [29]:
def check_proximity(content, query, expect):
    doc = mkdoc(content=content)
    query = parse_query(query)
    score = score_proximity(index_doc(doc), query)
    assert score == expect, f"{score}, {expect}"
    
check_proximity("a b c", "a b c", 0.0)  # FIXME

AssertionError: 2.475, 0.0